# Calculate Brightness

In [1]:
import math
from PIL import ImageStat, Image

In [2]:
def brightness(frame):
    frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    stat = ImageStat.Stat(frame)
    r,g,b = stat.mean
    return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

# Stream Data from Youtube

In [3]:
import pafy
import youtube_dl
import cv2
import numpy as np

In [4]:
def stream_data1(url):
    vPafy = pafy.new(url)
    play = vPafy.getbest(preftype="webm")

    #start the video
    cap = cv2.VideoCapture(play.url)
    frames = []
    brightness_vals = [] #avg brightness for each frame

    while (True):
        ret,frame = cap.read()
        if type(frame) == type(None):
            break
        frames.append(frame)
        brightness_vals.append(brightness(frame))
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break    

    cap.release()
    return brightness_vals, frames;

def stream_data2(url):
    vPafy = pafy.new(url)
    play = vPafy.getbest(preftype="webm")

    #start the video
    cap = cv2.VideoCapture(play.url)
    brightness_vals = [] #avg brightness for each frame

    while (True):
        ret,frame = cap.read()
        if type(frame) == type(None):
            break
        brightness_vals.append(brightness(frame))
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break    

    cap.release()
    return brightness_vals

In [5]:
url_list = ['https://www.youtube.com/watch?v=Q2c2mPpvcpw',
            'https://www.youtube.com/watch?v=J8E1g-cHV5M',
            'https://www.youtube.com/watch?v=hf9k3Xb5x6Y',
            'https://www.youtube.com/watch?v=1upXsnnPf2o',
            'https://www.youtube.com/watch?v=Q1IRVcPOXyY',
            'https://www.youtube.com/watch?v=Hq2SlCja3zo']
X_train =[]

for url in url_list:
    X_train = X_train + stream_data2(url)
    

# Prep Data for Model

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

In [9]:
scaler = StandardScaler()
X_train_scaled = np.array(X_train).reshape(-1, 1)
scaler.fit(X_train_scaled)
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename)

['scaler.save']

In [10]:
X_train_scaled = scaler.transform(X_train_scaled)

In [11]:
def make_windows(data, size):
    X = []
    i = 0
    while(i+size) <= len(data)-1:
        X.append(data[i:i+size])
        i += 1
        
    return X
    

In [12]:
X_train_win = make_windows(X_train_scaled, 24)

In [13]:
X_train_win = np.array(X_train_win).reshape(len(X_train_win), 24, 1)

# Build Model

In [14]:
import tensorflow as tf
from keras.layers import TimeDistributed, RepeatVector, LSTM, Dense, Dropout
from keras.models import Sequential

C:\Users\Adrian\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
model = Sequential()
model.add(LSTM(30, activation='relu', input_shape=(24,1)))
model.add(RepeatVector(24))
model.add(LSTM(30, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

In [16]:
model.fit(X_train_win, X_train_win, epochs = 64, verbose = 1)

Epoch 1/64
112803/112803 [==============================] - 54s 482us/step - loss: 0.0797
Epoch 2/64
112803/112803 [==============================] - 58s 511us/step - loss: 0.0199
Epoch 3/64
112803/112803 [==============================] - 58s 518us/step - loss: 0.0154
Epoch 4/64
112803/112803 [==============================] - 58s 518us/step - loss: 0.0130
Epoch 5/64
112803/112803 [==============================] - 60s 529us/step - loss: 0.0116
Epoch 6/64
112803/112803 [==============================] - 60s 536us/step - loss: 0.0103
Epoch 7/64
112803/112803 [==============================] - 60s 529us/step - loss: 0.0093
Epoch 8/64
112803/112803 [==============================] - 59s 519us/step - loss: 0.0084
Epoch 9/64
112803/112803 [==============================] - 59s 527us/step - loss: 0.0078
Epoch 10/64
112803/112803 [==============================] - 59s 527us/step - loss: 0.0069
Epoch 11/64
112803/112803 [==============================] - 59s 521us/step - loss: 0.0063
Epoch 12

KeyboardInterrupt: 

In [17]:
model.save_weights("C:/Users/Adrian/Desktop/Hackathons/Swamp/ae_weights_24_dos.h5")

In [ ]:
model.load_weights('C:/Users/Adrian/Desktop/Hackathons/Swamp/ae_weights_30_d20.h5')

In [104]:
porygon_brightness, porygon_frames = stream_data1('https://www.youtube.com/watch?v=Wu-KAY3lJ6A')

In [56]:
porygon_test = np.array(porygon_brightness).reshape(-1,1)
porygon_scale = scaler.transform(porygon_test)
porygon_win = make_windows(porygon_scale, 24)
porygon_input = np.array(porygon_win).reshape(len(porygon_win), 24, 1)

In [57]:
porygon_pred = model.predict(porygon_input)

In [115]:
mae_of_predictions = np.squeeze(np.max(np.square(porygon_pred-porygon_input), axis=1))
mae_threshold1 = np.mean(mae_of_predictions) + np.std(mae_of_predictions) 
print(np.mean(mae_of_predictions))# can use a running mean instead.
mae_threshold2 = np.mean(mae_of_predictions) - np.std(mae_of_predictions)
actual = np.where(np.logical_or(mae_of_predictions > mae_threshold1, mae_of_predictions < mae_threshold2))[0]

1.061517071301474


In [67]:
print(actual)

[ 84  85  86  87  88  89  90  91  92  93  94 104 301 302 303 304 305 306
 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 343 344
 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362
 363 364 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557
 558 559 560 561]


In [40]:
for i in range(24):
    print(porygon_input[420][i], porygon_pred[420][i])
print(mae_of_predictions[420], mae_threshold)

[-0.22921672] [-0.22640298]
[-0.25337897] [-0.17067342]
[-0.22403641] [-0.10625433]
[0.54613722] [-0.05452465]
[-0.22498841] [-0.00459923]
[-0.25499009] [0.02842946]
[-0.22673947] [0.00049862]
[0.53746912] [-0.05180921]
[-0.22827213] [-0.07695804]
[-0.23916762] [-0.08416488]
[-0.23509921] [-0.08321805]
[0.56499602] [-0.07728633]
[-0.2285638] [-0.06782146]
[-0.23438363] [-0.05741421]
[-0.23053253] [-0.04868452]
[-0.22752407] [-0.04154531]
[0.55719875] [-0.0373031]
[-0.23878229] [-0.03568994]
[-0.23473306] [-0.03769948]
[-0.22385171] [-0.04674431]
[0.54608119] [-0.07069524]
[-0.22615904] [-0.11477032]
[-0.26411246] [-0.18542643]
[-0.22326671] [-0.26040697]
0.4125266154113933 3.0983853442389746


In [87]:
import collections
from itertools import count
import pandas as pd

In [105]:
def moving_average(data, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(data, window, 'same')

def explain_anomalies(y, window_size, sigma=1.0):
    avg = moving_average(y, window_size).tolist()
    residual = y - avg
    # Calculate the variation in the distribution of the residual
    std = np.std(residual)
    return {'standard_deviation': round(std, 3),
            'anomalies_dict': collections.OrderedDict([(index, y_i) for
                                                       index, y_i, avg_i in zip(count(), y, avg)
              if (y_i > avg_i + (sigma*std)) | (y_i < avg_i - (sigma*std))])}

def explain_anomalies_rolling_std(y, window_size, sigma=1.0):
    avg = moving_average(y, window_size)
    avg_list = avg.tolist()
    residual = y - avg
    # Calculate the variation in the distribution of the residual
    testing_std = pd.rolling_std(residual, window_size)
    testing_std_as_df = pd.DataFrame(testing_std)
    rolling_std = testing_std_as_df.replace(np.nan,
                                  testing_std_as_df.ix[window_size - 1]).round(3).iloc[:,0].tolist()
    std = np.std(residual)
    return {'stationary standard_deviation': round(std, 3),
            'anomalies_dict': collections.OrderedDict([(index, y_i)
                                                       for index, y_i, avg_i, rs_i in zip(count(),
                                                                                           y, avg_list, rolling_std)
              if (y_i > avg_i + (sigma * rs_i))])}

In [106]:
anomalies = explain_anomalies_rolling_std(mae_of_predictions, 3)

C:\Users\Adrian\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
C:\Users\Adrian\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [107]:
x_anomaly = np.fromiter(anomalies['anomalies_dict'].keys(), dtype=int, count=len(anomalies['anomalies_dict']))
y_anomaly = np.fromiter(anomalies['anomalies_dict'].values(), dtype=float, count=len(anomalies['anomalies_dict']))

In [109]:
print(x_anomaly)

[  0   3   7  10  16  18  19  20  24  29  30  32  36  40  41  42  43  46
  47  48  50  53  54  57  58  59  76  80  84  93  94  95  96 104 113 117
 118 121 122 125 126 129 130 133 134 136 138 142 146 147 150 151 154 155
 161 166 171 177 180 185 188 190 191 192 193 196 197 200 207 208 212 220
 227 250 252 258 270 280 289 292 301 312 315 317 318 321 328 333 339 343
 345 354 355 362 374 377 383 390 399 402 408 413 417 421 426 427 429 436
 439 443 446 448 452 458 460 467 475 480 487 492 495 499 502 513 516 522
 526 529 535 536 537 548 552 559 561 567 572]


In [110]:
url = 'https://www.youtube.com/watch?v=Wu-KAY3lJ6A'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="webm")

    #start the video
cap = cv2.VideoCapture(play.url)
    
i = 0
while (True):
    ret,frame = cap.read()
    if type(frame) == type(None):
        break
    if(i > 420):
        cv2.imshow('frame', frame)
        cv2.waitKey(0)
            
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break    
    i+=1

cap.release()

KeyboardInterrupt: 

# Enhancing Video Frames

In [97]:
from PIL import ImageEnhance

In [98]:
def merge_windows(windows):
    ret = windows[0]
    
    for i in range(1, len(windows)):
        ret = np.append(ret, windows[i][9])
    print(ret)
    return ret
        
    


In [99]:
def enhance_video(real_brightness, predicted_brightness, anomaly_frame_chunk_ids, vid_frames, win_size):
    #convert chunk ids into their frames
    frames_to_edit = []
    
    for i in range(len(anomaly_frame_chunk_ids)-1):
        if(anomaly_frame_chunk_ids[i+1]-anomaly_frame_chunk_ids[i] < win_size):
            for j in range(anomaly_frame_chunk_ids[i+1]-anomaly_frame_chunk_ids[i]):
                frames_to_edit.append(anomaly_frame_chunk_ids[i]+j)
        else:
            for j in range(win_size):
                frames_to_edit.append(anomaly_frame_chunk_ids[i] + j)
    for i in range(win_size):
        frames_to_edit.append(anomaly_frame_chunk_ids[len(anomaly_frame_chunk_ids)-1] + i)
    
    real_brightness = merge_windows(real_brightness)
    predicted_brightness = merge_windows(predicted_brightness)
    
    for frame_num in frames_to_edit:
        factor = .4
        frame = Image.fromarray(cv2.cvtColor(vid_frames[frame_num], cv2.COLOR_BGR2RGB))
        enhancer = ImageEnhance.Brightness(frame)
        enhanced_frame = enhancer.enhance(factor)
        vid_frames[frame_num] = enhanced_frame
        
        
    return vid_frames

In [111]:
work = enhance_video(porygon_input, porygon_pred, x_anomaly, np.array(porygon_frames), 10)

#print(np.array(porygon_frames))

[ 0.46920981  0.47742069  0.49080899  0.49514151  0.51829399  0.52632538
  0.54079553  0.55398222  0.56189088  0.57794468  0.59060756 -1.30751293
 -1.31370917 -1.32304145 -1.32917991 -1.32793843 -1.32728905 -1.31861323
 -1.31894359 -1.32014348 -1.30867198 -1.31107405 -1.30982045 -1.30949366
  0.59060756 -1.30751293 -1.31370917 -1.32304145 -1.32917991 -1.32793843
 -1.32728905 -1.31861323 -1.31894359 -1.32014348 -1.30867198 -1.31107405
 -1.30982045 -1.30949366 -1.31248748 -1.3126035  -1.31194347 -1.33461151
 -1.33410063 -1.3282919  -1.33309263 -1.33504016 -1.3320605  -1.32159599
 -1.32713206 -1.3284849  -1.32481014 -1.32404517 -1.32283421 -1.28083927
 -1.28044877 -1.28047701 -1.28172622 -1.29566564 -1.30208393 -1.30285576
 -1.29631825 -1.30867832 -1.30898615 -1.30188143 -1.28452609 -1.29801465
 -1.29534116 -1.29826967 -1.2869775  -1.2971504  -1.2968291  -1.29922203
 -1.29905096 -1.29364676 -1.29245451 -1.39496315 -1.39970099 -1.40103115
 -1.40167738 -1.38158272 -1.38419125 -1.38374549 -1

In [112]:
for frame in work:
    cv2.imshow('frame', frame)
    cv2.waitKey(0)
            
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
print(porygon_frames)

In [ ]:
print(work)

In [ ]:
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize

In [ ]:
def make_video(frames, outimg=None, fps=24, size=None,
               is_color=True, format="MP4V"):

    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for frame in frames:
        if vid is None:
            if size is None:
                size = frame.shape[1], frame.shape[0]
            vid = VideoWriter('safe.mp4', fourcc, float(fps), size, is_color)
        if size[0] != frame.shape[1] and size[1] != frame.shape[0]:
            frame = resize(frame, size)
        vid.write(frame)
    vid.release()
    return vid

In [ ]:
make_video(porygon_frames)

ping


fuck me
{"destination": null}


In [15]:
import requests, os, json

In [16]:
REQUEST = json.dumps({
    'body': {}
})

In [17]:
# GET /test
request= json.loads(REQUEST)
res = {'sucess': 'nop'}
print(json.dumps(res))

{"sucess": "nop"}


In [18]:
# POST /test
req = json.loads(REQUEST)
url = req['body'].get('url')
print("fuck me")
res = {'destination': url}
print(json.dumps(res))

fuck me
{"destination": null}
